In [2]:
import requests
import pandas as pd
from pathlib import Path
r = requests.get("https://www.gapminder.org/documentation/documentation/gapdata010.xls")
assert r.ok
Path("data/input").mkdir(parents=True, exist_ok=True)
output = open("data/input/gapminder.xls","wb")
output.write(r.content)
output.close()

In [3]:
df = pd.read_excel("data/input/gapminder.xls")
df_d = df.drop(df.index[0:16]).reset_index()
df_d["Country"]=df_d["Country"].str.strip()
df_d = df_d[~pd.isna(df_d["MMR"])]

In [4]:
df_d.loc[:2, "year"] = ["1875", "1885", "1895"]
df_d.loc[
    (df_d["year"]==1772)
    & (df_d["Country"] == "Finland")
    & (df_d["Maternal deaths"] == 487),
    "year",
    ] = 1872
df_d.loc[
    (df_d["year"] == 1775)
    & (df_d["Country"] == "Finland")
    & (df_d["Maternal deaths"] == 629),
    "year"
    ] = 1875

df_d.loc[
    (df_d["year"] == 1967)
    & (df_d["Country"] == "Finland")
    & (df_d["Maternal deaths"] == 77),
    "year"
    ] = 1957

df_d.loc[
    (df_d["year"] == 1967)
    & (df_d["Country"] == "Sweden")
    & (df_d["Maternal deaths"] == 39),
    "year"
    ] = 1957

df_d.loc[
    (df_d["year"] == 1967)
    & (df_d["Country"] == "United States")
    & (df_d["Maternal deaths"] == 1766.28),
    "year"
    ] = 1957

df_d = df_d.drop(df_d[(df_d["MMR"] == 0) | (df_d["MMR"] == '...')].index)

df_d = df_d.drop(

    df_d[(df_d["Country"] == "New Zealand")
         & (df_d["year"]== 1950)
         & (df_d["MMR"] == 90)
         ].index
)

df_d = df_d.drop(
    df_d[
        (df_d["year"].isin(["1982-02", "1899-03"])) & (df_d["Country"] == "New Zealand")
    ].index

)
df_d.head()

,index,Country,year,MMR,Live Births,Maternal deaths,Women reproductive age (15-49),MM-rate,Unnamed: 7,Skilled attendance %,"Source MMR, maternal death and Live birth",Comment,Source women reproductive age,Comments
0,16,Australia,1875,643,NaN,NaN,NaN,NaN,NaN,NaN,"Loudon, Death in Childbirth,1992",only State of Virginia,NaN,NaN
1,17,Australia,1885,592,NaN,NaN,NaN,NaN,NaN,NaN,"Loudon, Death in Childbirth, 1992",only State of Virginia,NaN,NaN
2,18,Australia,1895,560,NaN,NaN,NaN,NaN,NaN,NaN,"Loudon, Death in Childbirth, 1992",only State of Virginia,NaN,NaN
3,19,Australia,1901-5,609,NaN,NaN,940332.0,0.0,NaN,NaN,"Loudon, Death in Childbirth, 1992",only State of Virginia,"Mitchell, B. International Historical Statisti...",NaN
4,20,Australia,1906,538,107890,NaN,NaN,NaN,NaN,NaN,"Loudon, Death in Childbirth, 1992",only State of Virginia,NaN,NaN


In [7]:

assert(df_d[df_d.duplicated(subset = ["Country", "year"], keep=False)].shape[0] == 0 )

In [8]:
df_check = df_d[['Country','year', 'Live Births', 'Maternal deaths', 'MMR']].dropna().reset_index(drop=True)
df_check['mmr_calc'] = ((df_check['Maternal deaths']/df_check['Live Births'])*100000).astype(float).round(2)
df_check['MMR'] = df_check['MMR'].astype(float).round(2)

df_check[ df_check['mmr_calc'] != df_check['MMR']]

,Country,year,Live Births,Maternal deaths,MMR,mmr_calc
597,Ireland,1901,71827.1584,1365.434281,618.0,1901.00
598,Ireland,1902,72580.042,1380.472399,623.0,1902.00
599,Ireland,1903,72616.1667,1381.885652,563.0,1903.00
600,Ireland,1904,74020.0312,1409.341394,562.0,1904.00
601,Ireland,1905,73242.8892,1395.277039,557.0,1905.00
602,Ireland,1906,73539.1845,1401.656857,586.0,1906.00
603,Ireland,1907,72435.2952,1381.341079,496.0,1907.00
604,Ireland,1908,72697.7941,1387.073911,520.0,1908.00
605,Ireland,1909,73043.1,1394.392779,546.0,1909.00
606,Ireland,1910,72697.7941,1388.527867,531.0,1910.00


In [12]:
import numpy as np
def get_mid_year(year: str):
    if "-" in str(year):
        year_begin = year.split("-")[0]
        year_end = year.split("-")[1]
        year_end_len = len(year.split("-")[1])
        year_end_pref = 4 - year_end_len
        if year_end_pref > 0:
            year_pref = year_begin[0:year_end_pref]
            year_end = year_pref + year_end
            year_out = int(np.mean([int(year_begin), int(year_end)]))
        else:
            year_out = int(np.mean([int(year_begin), int(year_end)]))
    else:
        year_begin = int(year)
        year_out = int(year)

    return pd.Series([year_begin, year_out])

In [13]:
df_years = df_d["year"].apply(get_mid_year)
df_d["year_begin"] = df_years[0].astype(int)
df_d["mid_year"] = df_years[1].astype(int)

df_d[df_d["year_begin"] > df_d["mid_year"]]

df_d.loc[
    (df_d["year"] == "1897-01") & (df_d["Country"] == "New Zealand"), "mid_year"
] = 1899
assert(df_d[df_d[["Country", "year"]].duplicated(keep=False)].shape[0] == 0)